In [24]:
!pip install PyMuPDF docx2txt pytesseract pillow pix2tex tk tkcalendar

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/40.9 kB ? eta -:--:--
     ------------------- ------------------ 20.5/40.9 kB 682.7 kB/s eta 0:00:01
     ---------------------------- --------- 30.7/40.9 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 40.9/40.9 kB 329.8 kB/s eta 0:00:00


In [25]:
import fitz
import docx2txt
import os
import pytesseract
from PIL import Image
from pix2tex.cli import LatexOCR
from IPython.display import display, Latex, Math
import tkinter as tk
from tkinter import simpledialog
from tkcalendar import Calendar
import requests
from datetime import datetime, timedelta
import jwt
import os
from dotenv import load_dotenv

# PDF Text and Image Extraction

In [5]:
doc = fitz.open(r"Report.pdf")

for page_num, page in enumerate(doc):
    # Extract text
    text = page.get_text()
    print(f"Text on Page {page_num + 1}:\n{text}\n")
    
    # Extract images
    image_list = page.get_images(full=True)
    for image_index, img in enumerate(image_list, start=1):
        xref = img[0]
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        image_filename = f"page_{page_num + 1}_image_{image_index}.png"
        with open(image_filename, "wb") as img_file:
            img_file.write(image_bytes)
        print(f"Saved Image: {image_filename}")

doc.close()  


Text on Page 1:
SCIENCE LAB REPORT
Lorem ipsum dolor sit amet, consectetuer adipiscing elit
Your Name
09.04.20XX
8TH GRADE SCIENCE


Saved Image: page_1_image_1.png
Text on Page 2:
INTRODUCTION
Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh
euismod tincidunt ut laoreet dolore magna aliquam erat volutpat. Ut wisi enim ad minim
veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nisl ut aliquip ex ea
commodo consequat.
HYPOTHESIS
Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat,
vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan.
MATERIALS
1. Lorem ipsum dolor sit amet
2. Consectetuer adipiscing elit
3. Sed diam nonummy nibh euismod
PROCEDURE
1. Lorem ipsum dolor sit amet
2. Consectetuer adipiscing elit
3. Sed diam nonummy nibh euismod
DATA
LOREM IPSUM
DOLOR SIT
Lorem ipsum
Lorem ipsum
Lorem ipsum
1


Text on Page 3:
RESULTS
Nam liber tempor cum soluta nobis eleifend option congue nihil

# Word Text and Image Extraction

In [6]:
# Specify the path to your Word document
doc_path = r"Report.docx"
# Specify the directory to save images
images_dir = r"images_directory"  # Adjust the path as needed

# Ensure the images directory exists
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

# Extract text and save images from the document
text = docx2txt.process(doc_path, images_dir)

# Print the extracted text
print(text)

SCIENCE LAB REPORT

Lorem ipsum dolor sit amet, consectetuer adipiscing elit



Your Name

09.04.20XX

8TH GRADE SCIENCE









INTRODUCTION

Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat. Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nisl ut aliquip ex ea commodo consequat.

HYPOTHESIS

Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat, vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan.

MATERIALS

Lorem ipsum dolor sit amet

Consectetuer adipiscing elit

Sed diam nonummy nibh euismod

PROCEDURE

Lorem ipsum dolor sit amet

Consectetuer adipiscing elit

Sed diam nonummy nibh euismod

DATA

LOREM IPSUM

DOLOR SIT



Lorem ipsum





Lorem ipsum





Lorem ipsum





RESULTS

Nam liber tempor cum soluta nobis eleifend option congue nihil imperdiet doming id quod mazim placerat facer possim as

# OCR Math Equation Extraction

In [20]:
img = Image.open(r"C:\Users\arnav\Downloads\Screenshot 2024-02-17 085808.png")
model = LatexOCR()

# Your LaTeX equation as a Python string
latex_str = model(img)

# Display the LaTeX equation
display(Math(latex_str))

<IPython.core.display.Math object>

# Calendly Embedding + Zoom Embedding

In [ ]:
# Load environment variables
load_dotenv()

API_KEY = os.getenv('ZOOM_API_KEY')
API_SEC = os.getenv('ZOOM_API_SECRET')

In [ ]:
# Function to generate JWT token
def generate_token():
    payload = {
        'iss': API_KEY,
        'exp': datetime.now() + timedelta(days=2)
    }
    return jwt.encode(payload, API_SEC, algorithm='HS256')

# Function to create a Zoom meeting
def create_zoom_meeting(meeting_date):
    headers = {
        'authorization': f'Bearer {generate_token()}',
        'content-type': 'application/json'
    }
    data = {
        'topic': 'My Meeting',
        'type': 2,
        'start_time': meeting_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
        'duration': 40,
        'timezone': 'UTC',
        'agenda': 'Test meeting',
        'settings': {
            'host_video': 'false',
            'participant_video': 'false',
            'join_before_host': True,
        }
    }
    response = requests.post('https://api.zoom.us/v2/users/me/meetings', headers=headers, json=data)
    if response.status_code == 201:
        meeting_info = response.json()
        tk.messagebox.showinfo(title="Meeting Created", message=f"Meeting ID: {meeting_info['id']}\nJoin URL: {meeting_info['join_url']}")
    else:
        tk.messagebox.showerror(title="Error", message="Failed to create meeting")

# Function to handle date selection
def on_date_select():
    selected_date = calendar.selection_get()
    selected_time = simpledialog.askstring("Input", "Enter meeting time (HH:MM):", parent=root)
    if selected_time:
        try:
            meeting_datetime = datetime.strptime(f"{selected_date} {selected_time}", "%Y-%m-%d %H:%M")
            create_zoom_meeting(meeting_datetime)
        except ValueError:
            tk.messagebox.showerror("Error", "Invalid time format. Please use HH:MM.")

In [ ]:
root = tk.Tk()
root.title("Zoom Meeting Scheduler")

calendar = Calendar(root, selectmode='day')
calendar.pack(pady=20)

schedule_button = tk.Button(root, text="Schedule Meeting", command=on_date_select)
schedule_button.pack(pady=20)

root.mainloop()